In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from plotnine import *

In [2]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "hw3_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.executor.memory","6g")
sc = spark.sparkContext

In [3]:
# def convertSAStoCSV(directory,excludes=[]):
#     df=None
#     for filename in os.listdir(directory):
#         if not filename.endswith('.zip') and filename not in excludes:
#                 df=pd.read_sas(os.path.join(directory,filename))
#                 file,ext=os.path.splitext(os.path.join(directory,filename))
#                 df.to_csv(os.path.join(file+'.csv'),index=False)
# convertSAStoCSV('data/2014',['formats.sas7bcat','Format14.sas'])

In [6]:
def getdata(directory,excludes=[]):
    df=None
    for filename in os.listdir(directory):
        if filename.endswith('.csv') and filename not in excludes:
            file,ext=os.path.splitext(filename)
            dftmp=spark.read.csv(os.path.join(directory,filename),header=True)
            for col in dftmp.columns:
                if col!='ST_CASE':
                    dftmp=dftmp.withColumnRenamed(col,file+ '_' + col)
            if df is None:
                df=dftmp
            else:
                df=df.join(dftmp,on='ST_CASE',how='full')
    return df

In [7]:
df=getdata('data/2014')

In [8]:
df.show()

Py4JJavaError: An error occurred while calling o827.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 131 in stage 41.0 failed 1 times, most recent failure: Lost task 131.0 in stage 41.0 (TID 1396, localhost, executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.copy(UnsafeRow.java:504)
	at org.apache.spark.sql.catalyst.expressions.UnsafeRow.copy(UnsafeRow.java:62)
	at org.apache.spark.sql.execution.joins.SortMergeFullOuterJoinScanner.findMatchingRows(SortMergeJoinExec.scala:1031)
	at org.apache.spark.sql.execution.joins.SortMergeFullOuterJoinScanner.advanceNext(SortMergeJoinExec.scala:1121)
	at org.apache.spark.sql.execution.joins.FullOuterIterator.advanceNext(SortMergeJoinExec.scala:1141)
	at org.apache.spark.sql.execution.RowIteratorToScala.hasNext(RowIterator.scala:68)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11

565

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kchuangk/.pyenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-3c9a60fd698f>", line 1, in <module>
    df.count()
  File "/home/kchuangk/.pyenv/lib/python3.6/site-packages/pyspark/sql/dataframe.py", line 522, in count
    return int(self._jdf.count())
  File "/home/kchuangk/.pyenv/lib/python3.6/site-packages/py4j/java_gateway.py", line 1284, in __call__
    answer = self.gateway_client.send_command(command)
  File "/home/kchuangk/.pyenv/lib/python3.6/site-packages/py4j/java_gateway.py", line 1014, in send_command
    response = connection.send_command(command)
  File "/home/kchuangk/.pyenv/lib/python3.6/site-packages/py4j/java_gateway.py", line 1181, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
Keyboa

KeyboardInterrupt: 